In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import json
from torchvision import transforms
from sklearn.model_selection import train_test_split
from create_dataset import generate_circle_dataset

In [4]:
%pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 MB 5.1 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2
Note: you may need to restart the kernel to use updated packages.


In [ ]:


# --------- Dataset Definition ---------
class CircleDataset(Dataset):
    def __init__(self, data_path):
        self.files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith('.json')]
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        with open(self.files[idx], 'r') as f:
            data = json.load(f)
        img = np.array(data['img'], dtype=np.float32)
        label = data['label']
        img = np.expand_dims(img, axis=0)  # Add channel dimension
        target = torch.tensor([label['row'], label['col'], label['radius']], dtype=torch.float32)
        return torch.tensor(img, dtype=torch.float32), target

# --------- CNN Model Definition ---------
class CircleFinderCNN(nn.Module):
    def __init__(self):
        super(CircleFinderCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 3)  # Output: row, col, radius

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 16, 128, 128]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 32, 64, 64]
        x = x.view(-1, 32 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# --------- Training Loop ---------
def train_model(model, train_loader, val_loader, device, epochs=10, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for imgs, targets in train_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for imgs, targets in val_loader:
                imgs, targets = imgs.to(device), targets.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

# --------- Example Usage ---------
if __name__ == '__main__':
    # Generate data if needed
    data_dir = './circle_data'
    os.makedirs(data_dir, exist_ok=True)
    generate_circle_dataset(data_dir, avg_noise=2, n=1000)

    # Load dataset
    full_dataset = CircleDataset(data_dir)
    train_idx, val_idx = train_test_split(list(range(len(full_dataset))), test_size=0.2)
    train_dataset = torch.utils.data.Subset(full_dataset, train_idx)
    val_dataset = torch.utils.data.Subset(full_dataset, val_idx)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16)

    # Initialize and train model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = CircleFinderCNN()
    train_model(model, train_loader, val_loader, device, epochs=10)
